In [ ]:
#!python -m pip install --user -r ../requirements.txt

In [ ]:
!python -m pip install --user docx2pdf pdf2image

In [ ]:
PATH_TO_DOCX="./"

In [ ]:
# Windows Only section
from docx2pdf import convert
import os

for file in os.listdir(PATH_TO_DOCX):
    try:
        convert(file)
    except:
        print(f"bug with {file}")

In [ ]:
import pdf2image


import os

if "OS" in os.environ.keys() and "WINDOWS" in os.environ["OS"].upper():
    OS="WINDOWS"

    POPPLER_PATH = r"C:\Users\conrad.thiounn\Desktop\poppler-23.01.0\Library\bin"
else:
    OS="UNIX"

    POPPLER_PATH = r"/usr/share/poppler"

In [ ]:
for file in os.listdir(PATH_TO_DOCX):
    if file.endswith(".pdf"):
        try:
            pdf2image.convert_from_path(pdf_path=file,output_folder=PATH_TO_DOCX,thread_count=os.cpu_count(),output_file=file.replace(".pdf",""),fmt="jpeg")
        except:
            print(f"bug with {file}")